# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment : IoT and Edge Devices

## Learning Objectives

At the end of the experiment, you will be able to

* create TensorFlowLite model and TensorFlowLite Micro model for the MNIST data



## Setup Steps:

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2236974" #@param {type:"string"}

In [2]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "7259357831" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()

notebook= "M3_AST_33_MNIST_DNN_TensorFlow_Lite_Micro_C" #name of the notebook

def setup():
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/mnist_train.zip")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/mnist_test.zip")
    ipython.magic("sx unzip mnist_train.zip")
    ipython.magic("sx unzip mnist_test.zip")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError
    else:
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError
    else:
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


### Importing required packages

In [ ]:
import os
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
# Set seed for experiment reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Define paths to model files
# Creating the MNIST_DIGIT_EXP directory to move the csv files to it
if not os.path.exists('MNIST_DIGIT_EXP'):
    os.mkdir('MNIST_DIGIT_EXP')

MODELS_DIR = 'MNIST_DIGIT_EXP/models'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + '/tensor_flowmodel'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + '/model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + "/quant_model.tflite"
MODEL_TFLITE_MICRO_QUANT = MODELS_DIR + '/quant_model.cc'
MODEL_TFLITE_MICRO_NO_QUANT = MODELS_DIR + '/no_quant_model.cc'

### Loading MNIST Data

In [ ]:
train_dataset = pd.read_csv('/content/mnist_train.csv' , header=None)
test_dataset = pd.read_csv('/content/mnist_test.csv' , header=None)

In [ ]:
# Converting the pandas dataframe to numpy array
traindatasetnp = np.array(train_dataset)
testdatasetnp = np.array(test_dataset)

In [ ]:
# The shape attribute for numpy arrays returns the dimensions of the array
# For example, if Y has n rows and m columns, then Y.shape is (n,m). So Y.shape[0] is n
print(traindatasetnp.shape[0], traindatasetnp.shape[1]);

60000 785


In [ ]:
# Make a copy of the train and test sets with datasize and 784 pixel values
train_data = traindatasetnp[:,1:traindatasetnp.shape[1]].copy();
test_data = testdatasetnp[:,1:testdatasetnp.shape[1]].copy();

In [ ]:
# Check for the shape
print(train_data.shape)
print(test_data.shape)

(60000, 784)
(10000, 784)


In [ ]:
# Make a copy of the train and test labels from the numpy array
train_labels = traindatasetnp[:,0].copy();
test_labels = testdatasetnp[:,0].copy();

In [ ]:
# Check for the shape of labels
print(train_labels.shape)
print(test_labels.shape)

(60000,)
(10000,)


In [ ]:
# Converting train and test data to float32 array
train_data_scaled = np.array(train_data, np.float32)
test_data_scaled = np.array(test_data, np.float32)

In [ ]:
print(train_data_scaled[1])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.  51. 159. 253. 159.  50.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  48. 238. 252. 252. 252. 237.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  54.
 227. 253. 252. 239. 233. 252.  57.   6.   0.   0.   0.   0.   0.   0.
   0. 

In [ ]:
# Make a copy of the float converted data
train_data = train_data_scaled.copy()
test_data = test_data_scaled.copy()

### Visualizing the image


In [ ]:
# 255 corresponds to white data, the closer it is to 255 it will correspond to a white pixel
# the closer it is to zero it wil correspond to black pixels
# Reshaping 784 to 28*28
# from train_data taking 369 row for visualization
cv2_imshow(255*(train_data[369].reshape(28,28)))

### Label Encoding

In [ ]:
# Label encoding for train data
traindata = (train_labels.size, train_labels.max()+1)
print(traindata)

(60000, 10)


In [ ]:
# one hot encode the train labels
one_hot_train_labels = np.zeros(traindata)

In [ ]:
rows = np.arange(train_labels.size)
print(rows.shape)

(60000,)


In [ ]:
one_hot_train_labels[rows, train_labels] = 1

In [ ]:
print(train_labels)
print(one_hot_train_labels)
print(one_hot_train_labels.shape)

[5 0 4 ... 5 6 8]
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
(60000, 10)


In [ ]:
# Label encoding for test data
testdata = (test_labels.size, test_labels.max()+1)
one_hot_test_labels = np.zeros(testdata)

In [ ]:
rows = np.arange(test_labels.size)
one_hot_test_labels[rows, test_labels] = 1
print(one_hot_test_labels)
print(one_hot_test_labels.shape)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(10000, 10)


### Create Model

In [ ]:
model = tf.keras.Sequential()

# Only takes one feature
# Input of 784
model.add(keras.layers.Dense(100, activation='relu', input_shape=(784,)))
model.add(keras.layers.Dense(100, activation='relu'))
# Only one output
model.add(keras.layers.Dense(10))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=["mae"]) # tf.keras.losses.BinaryCrossentropy()

We can also go for higher number of nodes also for better accuracy and for better convergence, but we have to finally put in the hardware so need to make sure that we compress it as much as possible in the training phase itself. So that there is not much loss when we perform quantization and pruning.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 89610 (350.04 KB)
Trainable params: 89610 (350.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
history = model.fit(train_data, one_hot_train_labels, epochs=500, batch_size=600,
                    validation_data=(test_data, one_hot_test_labels))

Epoch 1/500
100/100 [==============================] - 7s 6ms/step - loss: 226.1834 - mae: 8.1612 - val_loss: 12.6947 - val_mae: 2.5617
Epoch 2/500
100/100 [==============================] - 0s 4ms/step - loss: 6.8069 - mae: 1.6713 - val_loss: 3.2518 - val_mae: 1.0305
Epoch 3/500
100/100 [==============================] - 0s 4ms/step - loss: 2.2677 - mae: 0.7916 - val_loss: 1.4636 - val_mae: 0.5822
Epoch 4/500
100/100 [==============================] - 0s 4ms/step - loss: 1.1726 - mae: 0.4982 - val_loss: 0.8833 - val_mae: 0.4064
Epoch 5/500
100/100 [==============================] - 0s 4ms/step - loss: 0.7499 - mae: 0.3688 - val_loss: 0.6162 - val_mae: 0.3191
Epoch 6/500
100/100 [==============================] - 0s 4ms/step - loss: 0.5384 - mae: 0.2983 - val_loss: 0.4700 - val_mae: 0.2695
Epoch 7/500
100/100 [==============================] - 0s 4ms/step - loss: 0.4164 - mae: 0.2558 - val_loss: 0.3798 - val_mae: 0.2378
Epoch 8/500
100/100 [==============================] - 0s 4ms/step

In [ ]:
# Save the model to disk
model.save(MODEL_TF)

In [ ]:
# Evaluate the model
test_loss, test_mae = model.evaluate(test_data_scaled, one_hot_test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0608 - mae: 0.1258


In [ ]:
# Get the model predictions on test data
y_test_pred = model.predict(test_data)

313/313 [==============================] - 1s 1ms/step


In [ ]:
# Visualize and print the actual and predicted image
cv2_imshow(255*test_data[163].reshape(28,28))
np.argmax(y_test_pred[163])

4

In [ ]:
x_test = tf.convert_to_tensor(train_data, dtype=tf.float32)
def representative_dataset():
  for i in range(300):
    yield([tf.reshape(x_test[i],(1, 784))])

### Convert the model to TFLite without quantization

In [ ]:
# Convert the model to TFLite without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

360320

### Convert and save the model with quantization

In [ ]:
# Convert and save the model with quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  #TFLITE_BUILTINS
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Create and provide a representative dataset
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()
open(MODEL_TFLITE, "wb").write(model_tflite)

92480

The tflite model has 92432 total number of parameters.

In [ ]:
# TFLite with no quant
x_test = test_data.copy()
# Converting test data to float32 array
x_test = x_test.astype(np.float32)

x_labels = one_hot_test_labels.copy()
# Converting the test labels to float32 array
x_labels = x_labels.astype(np.float32)

In [ ]:
# Initialize and allocate memory without quantization
no_q_interpreter = tf.lite.Interpreter(model_content=model_no_quant_tflite)

# Load the TFLite model and allocate tensors
no_q_interpreter.allocate_tensors() # Needed before execution!

In [ ]:
# Get input tensors.
no_q_interpreter.get_input_details()

[{'name': 'serving_default_dense_input:0',
  'index': 0,
  'shape': array([  1, 784], dtype=int32),
  'shape_signature': array([ -1, 784], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
# Get output tensors
no_q_interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 9,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
y_test_pred_no_quant_tflite = np.empty(x_labels.shape, dtype=np.float32)
print(y_test_pred_no_quant_tflite.shape)

(10000, 10)


In [ ]:
# Check the predictions on the tensorflow lite model on test data
for i in range(len(x_test)):
    input_shape = no_q_interpreter.get_input_details()[0]['shape'];
    inputtensor = np.array(x_test[i].reshape(input_shape), dtype=np.float32)
    no_q_interpreter.set_tensor(no_q_interpreter.get_input_details()[0]["index"], inputtensor)
    # Run inference
    no_q_interpreter.invoke()
    y_test_pred_no_quant_tflite[i,:] = no_q_interpreter.get_tensor(no_q_interpreter.get_output_details()[0]["index"])[0]

cv2_imshow(x_test[12].reshape(28,28))
print(np.argmax(y_test_pred_no_quant_tflite[12]))

9


### TensorFLow Lite Micro

TensorFLow Lite still operates in floating point, now micro controllers, let's say embedded controllers, which supports floating point operation, can put the tensorflow lite into those microcontrollers. But there are some microcontrollers which does not support floating point hardware, they might support floating point software, for the hardware which doesn't support floating point for that we need to reduce the size which convert this floating point 32 bit data into a 8 bit integer data and then we could use these models which are approximately 1/4 size of the TensorFlow lite model and then we can put these models into microcontrollers.

In [ ]:
x_test = test_data.copy()
x_test = x_test.astype(np.float32)
x_labels = one_hot_test_labels.copy()
x_labels = x_labels.astype(np.int8)

In [ ]:
# Initialize and allocate memory with quantization
q_interpreter = tf.lite.Interpreter(model_content=model_tflite)
q_interpreter.allocate_tensors()

In [ ]:
# Get the input and output tensors
q_interpreter.get_input_details()
q_interpreter.get_output_details()

input_details = q_interpreter.get_input_details()[0]
output_details = q_interpreter.get_output_details()[0]

In [ ]:
input_details

{'name': 'serving_default_dense_input:0',
 'index': 0,
 'shape': array([  1, 784], dtype=int32),
 'shape_signature': array([ -1, 784], dtype=int32),
 'dtype': numpy.int8,
 'quantization': (1.0, -128),
 'quantization_parameters': {'scales': array([1.], dtype=float32),
  'zero_points': array([-128], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [ ]:
output_details

{'name': 'StatefulPartitionedCall:0',
 'index': 9,
 'shape': array([ 1, 10], dtype=int32),
 'shape_signature': array([-1, 10], dtype=int32),
 'dtype': numpy.int8,
 'quantization': (0.005541609600186348, -99),
 'quantization_parameters': {'scales': array([0.00554161], dtype=float32),
  'zero_points': array([-99], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [ ]:
input_scale, input_zero_point = input_details["quantization"]

x_test_ = x_test / input_scale + input_zero_point

x_test_ = x_test_.astype(input_details["dtype"])
print(x_test_.shape)

(10000, 784)


In [ ]:
# Invoke the interpreter with quantization
y_test_pred_tflite = np.empty(x_labels.shape, dtype=output_details["dtype"])
for i in range(len(x_test_)):
    input_shape= q_interpreter.get_input_details()[0]['shape'];
    inputtensor = np.array(x_test_[i].reshape(input_shape), dtype=np.int8)
    q_interpreter.set_tensor(input_details["index"], inputtensor)
    q_interpreter.invoke()
    y_test_pred_tflite[i,:] = q_interpreter.get_tensor(output_details["index"])[0]

output_scale, output_zero_point = output_details["quantization"]
y_test_pred_tflite = y_test_pred_tflite.astype(np.float32)
y_test_pred_tflite1 = (y_test_pred_tflite - output_zero_point) * output_scale

cv2_imshow(x_test[12].reshape(28,28))
print(np.argmax(y_test_pred_tflite[12]))

print(x_test_[12])

7
[-128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128 -128
 -12

In [ ]:
#!apt-get update && apt-get -qq install xxd
!apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO_QUANT}
# Print the C source file
!cat {MODEL_TFLITE_MICRO_QUANT}

Streaming output truncated to the last 5000 lines.
  0xff, 0xfe, 0xfc, 0xfd, 0x00, 0xfd, 0xfe, 0xfc, 0x05, 0xfb, 0x01, 0xfb,
  0x05, 0xfe, 0xfe, 0xfe, 0x03, 0x00, 0xfd, 0xfb, 0xfb, 0xff, 0xfd, 0x04,
  0x01, 0x00, 0x01, 0x01, 0xfb, 0xfe, 0x05, 0x03, 0x03, 0xff, 0x05, 0xff,
  0x04, 0x02, 0x03, 0xfc, 0xfd, 0xfe, 0xfd, 0x04, 0xfd, 0xfc, 0xfe, 0x02,
  0xfc, 0xfa, 0x00, 0x00, 0xfa, 0x01, 0x00, 0x03, 0x00, 0xfb, 0xfb, 0x01,
  0xfc, 0xff, 0xfd, 0xff, 0xfe, 0xfd, 0x03, 0x03, 0x00, 0xff, 0xfe, 0xfe,
  0xfe, 0xfe, 0x05, 0xfd, 0x01, 0x03, 0x02, 0xfc, 0xfc, 0xfe, 0x02, 0xfb,
  0x03, 0x03, 0xfc, 0xfc, 0xfd, 0x01, 0x04, 0xff, 0xfb, 0x05, 0xfe, 0x03,
  0xff, 0x04, 0x06, 0x03, 0x04, 0xff, 0xfd, 0x02, 0x03, 0x02, 0x04, 0x02,
  0xfc, 0xfe, 0xfb, 0xfb, 0xfe, 0x02, 0x04, 0xfd, 0xfe, 0x02, 0xfb, 0xfc,
  0x04, 0xff, 0xfb, 0x02, 0xfe, 0x04, 0xfb, 0xfe, 0xfb, 0x03, 0x00, 0xff,
  0xfd, 0x04, 0x01, 0xff, 0xfd, 0x01, 0xff, 0x05, 0x04, 0x04, 0x03, 0xfb,
  0xfe, 0xfb, 0xfc, 0xff, 0xfb, 0x03, 0x04, 0xfe, 0xfb, 0xfc,

In [ ]:
#!apt-get update && apt-get -qq install xxd
!apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_NO_QUANT_TFLITE} > {MODEL_TFLITE_MICRO_NO_QUANT}
# Print the C source file
!cat {MODEL_TFLITE_MICRO_NO_QUANT}

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Another Way of getting the array in .h file
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [ ]:
c_model_name = "model_tflite_no_quant"
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(model_no_quant_tflite , c_model_name)) # model_tflite

In [ ]:
c_model_name = "model_tflite_quant"
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(model_tflite, c_model_name))

. h files are header files that are prewritten for our compiler. .cc files are our C source code files in which our code is written and are created by the user.

In [ ]:
input = np.array([test_data[145,:]]);
print((input.dtype))

float32


In [ ]:
y_test_pred_single = model.predict(input)
cv2_imshow(255*input.reshape(28,28))
np.argmax(y_test_pred_single)

1/1 [==============================] - 0s 35ms/step


8

### Please answer the questions below to complete the experiment:




TensorFlow Lite is a set of tools that enables on-device machine learning by helping developers run their models on mobile, embedded, and IoT devices. One of the key features is, it is optimized for on-device machine learning, by addressing 5 key constraints:


A. **Latency** : There's no round-trip to a server  

B. **Privacy** : No personal data leaves the device

C. **Connectivity** : Internet connectivity is not required

D. **Size** : Reduced model and binary size

E. **Power consumption** : Efficient inference and a lack of network connections



In [4]:
#@title Q.1. Which of the above statement(s) about TensorFlow Lite is/are True?
Answer1 = "All of the above" #@param ["","Only A and B","Only B and D","Only C and D", "Only D and E", "All of the above","None of the above"]


#### Consider the following statements given below and answer Q2.

A. It is possible to convert a TensorFlow model into a TensorFlow Lite model by using TensorFlow Lite Converter.

B. During conversion (TensorFlow model into a TensorFlow Lite model), we can apply optimizations such as quantization to reduce model size and latency with minimal or no loss in accuracy.





In [5]:
#@title Q.2. Which of the above statement(s) is/are True?
Answer2 = "Both A and B" #@param ["","Only A", "Only B", "Both A and B", "Neither A nor B"]


In [6]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [7]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "None" #@param {type:"string"}


In [8]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [9]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [10]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [11]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 2774
Date of submission:  24 Sep 2023
Time of submission:  16:54:04
View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions
